In [1]:
#import required libraries
!pip install opencv-python
!pip install tracker

In [2]:
import cv2
import csv
import collections
import numpy as np
from tracker import *

from itertools import zip_longest

In [3]:
import math

class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h, index = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    # print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id, index])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count, index])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id, index = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids



def ad(a, b):
    return a+b

In [4]:
# Initialize Tracker
tracker = EuclideanDistTracker()

# Initialize the videocapture object
cap = cv2.VideoCapture('BS.AVI')
input_size = 320

# Detection confidence threshold
confThreshold = 0.2
nmsThreshold = 0.2

font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 2

# Middle cross line position
middle_line_position = 500  
up_line_position = middle_line_position - 10
down_line_position = middle_line_position + 10

# List for store vehicle count information
temp_up_list = []
temp_down_list = []
vehicle_list =[]
counts = [0, 0, 0, 0, 0]
up_list = [0, 0, 0, 0, 0]
down_list = [0, 0, 0, 0, 0]

In [5]:
# Store Coco Names in a list
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)
print(len(classNames))

autoFile = "auto.names"
autoNames = open(autoFile).read().strip().split('\n')
print(autoNames)
print(len(autoNames))



# class index for our required detection classes
required_class_index = [2, 3, 5, 7]
auto_class_index = [0]

detected_classNames = []

## Model Files
modelConfiguration = 'yolov3-320.cfg'
modelWeigheights = 'yolov3-320.weights'

autoConfiguration = 'yolov3_custom.cfg'
autoWeigheights = 'yolov3_custom_2000.weights'

# configure the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

net1 = cv2.dnn.readNetFromDarknet(autoConfiguration, autoWeigheights)

# Configure the network backend

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

net1.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net1.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

np.random.seed(42)
colorsauto = np.random.randint(0, 255, size=(len(autoNames), 3), dtype='uint8')

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80
['Auto']
1


In [6]:
# Function for finding the center of a rectangle
def find_center(x, y, w, h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy=y+y1
    return cx, cy

In [7]:

# Function for count vehicle
def count_vehicle(box_id, img):

    x, y, w, h, id, index = box_id

    # Find the center of the rectangle for detection
    center = find_center(x, y, w, h)
    ix, iy = center
#single
#     if id not in vehicle_list:
#         vehicle_list.append(id)
#         counts[index]+=1
    
    if (iy > up_line_position) and (iy < middle_line_position):
        if id not in temp_up_list:
            temp_up_list.append(id)
            print("tup",id,temp_up_list)
            up_list[index] = up_list[index]+1

    elif (iy < down_line_position) and (iy > middle_line_position):
        if id not in temp_down_list:
            temp_down_list.append(id)
            print("tdown",id,temp_down_list)
            down_list[index] = down_list[index] + 1
            
    elif iy < up_line_position:
        if id in temp_down_list:
            temp_down_list.remove(id)
            print("up",id,index)
            up_list[index] = up_list[index]+1

    elif iy > down_line_position:
        if id in temp_up_list:
            temp_up_list.remove(id)
            print("down",id,index)
            down_list[index] = down_list[index] + 1

    # Draw circle in the middle of the rectangle
    cv2.circle(img, center, 2, (0, 0, 255), -1)  # end here
#     print(up_list, down_list)

In [8]:
# Function for finding the detected objects from the network output
def postProcess(outputs,img,auto):
    global detected_classNames 
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if not auto:
                if classId in required_class_index:
                    if confidence > confThreshold:
#                         print(classId,"vehicle")
                        w,h = int(det[2]*width) , int(det[3]*height)
                        x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
                        boxes.append([x,y,w,h])
                        classIds.append(classId)
                        confidence_scores.append(float(confidence))
            else:
                if classId in auto_class_index:
                    if confidence > confThreshold:
#                         print(classId,"auto")
                        w,h = int(det[2]*width) , int(det[3]*height)
                        x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
                        boxes.append([x,y,w,h])
                        classIds.append(classId)
                        confidence_scores.append(float(confidence))
                

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
    # print(classIds)
    if len(indices)>0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            # print(x,y,w,h)
            
            if not auto:
                color = [int(c) for c in colors[classIds[i]]]
                name = classNames[classIds[i]]
#                 print(name)
                detected_classNames.append(name)
                
                # Draw classname and confidence score 
                cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
                          (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

                # Draw bounding rectangle
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
                detection.append([x, y, w, h, required_class_index.index(classIds[i])])

            else:
                colorauto = [int(c) for c in colorsauto[classIds[i]]]
                name = autoNames[classIds[i]]
#                 print(name)
                detected_classNames.append(name)


                    # Draw classname and confidence score 
                cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
                          (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colorauto, 1)

                # Draw bounding rectangle
                cv2.rectangle(img, (x, y), (x + w, y + h), colorauto, 1)
                detection.append([x, y, w, h, len(required_class_index) + auto_class_index.index(classIds[i])])
#                 print("auto",[x, y, w, h, len(required_class_index) + auto_class_index.index(classIds[i])])
                
            # Update the tracker for each object
        boxes_ids = tracker.update(detection)
#         print("box_ids",boxes_ids)
        for box_id in boxes_ids:
#             print("id",box_id)
            count_vehicle(box_id, img)        
                
#             # Draw classname and confidence score 
#             cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
#                       (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

#             # Draw bounding rectangle
#             cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
#             detection.append([x, y, w, h, required_class_index.index(classIds[i])])


In [9]:
def realTime():
    while True:
        success, img = cap.read()
        try:
            img = cv2.resize(img,(0,0),None,0.5,0.5)
            ih, iw, channels = img.shape
            blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

            # Set the input of the network
            net1.setInput(blob)
            layersNames1 = net1.getLayerNames()
#             print(layersNames)
            outputNames1 = [(layersNames1[i- 1]) for i in net1.getUnconnectedOutLayers()]
#             print(outputNames)
            # Feed data to the network
            outputs1 = net1.forward(outputNames1)

            # Find the objects from the network output
            postProcess(outputs1,img,True)
            
            
            
            # Set the input of the network
            net.setInput(blob)
            layersNames = net.getLayerNames()
#             print(layersNames)
            outputNames = [(layersNames[i- 1]) for i in net.getUnconnectedOutLayers()]
#             print(outputNames)
            # Feed data to the network
            outputs = net.forward(outputNames)

            # Find the objects from the network output
            postProcess(outputs,img,False)
            
            # Draw the crossing lines
            cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 2)
            cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 2)
            cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 2)

            # Draw counting texts in the frame
            cv2.putText(img, "Count", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#          cv2.putText(img, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Car:        "+ str(down_list[0]), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Motorbike:  "+ str(down_list[1]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Bus:        "+ str(down_list[2]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Truck:      "+ str(down_list[3]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            cv2.putText(img, "Auto:       "+ str(down_list[4]), (20, 120), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

#single lane
#             cv2.putText(img, "Car:        "+str(counts[0]),(20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#             cv2.putText(img, "Motorbike:  "+str(counts[1]),(20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#             cv2.putText(img, "Bus:        "+str(counts[2]),(20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#             cv2.putText(img, "Truck:      "+str(counts[3]),(20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#             cv2.putText(img, "Auto:      "+str(counts[4]), (20, 120),cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
            
            
            
            # Show the frames
            cv2.imshow('Output', img)

            if cv2.waitKey(1) == ord('q'):
                break
        except Exception as e:
            print(str(e))
            
    # realese the capture object and destroy all active windows
    cap.release()
    cv2.destroyAllWindows()



In [ ]:
if __name__ == '__main__':
    realTime()

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

tup 223 [223]
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

tup 416 [223, 416]
tup 431 [223, 416, 431]
OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize

tdown 6529 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529]
tdown 6539 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539]
tdown 6549 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549]
tdown 6562 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562]
tdown 6576 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576]
tdown 6586 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372

tdown 6870 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870]
tdown 6879 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879]
tdown 6886 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763

tdown 7035 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 6967, 6975, 6981, 6988, 6995, 7005, 7013, 7027, 7035]
tdown 7044 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 6967, 6975, 6981, 6988, 6995, 7005, 7013, 7027, 7035, 7044]
tdown 7050 [622, 819, 1850, 2359, 2914

tdown 7176 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 6967, 6975, 6981, 6988, 6995, 7005, 7013, 7027, 7035, 7044, 7050, 7059, 7071, 7079, 7091, 7101, 7110, 7121, 7131, 7138, 7146, 7153, 7160, 7168, 7176]
tdown 7182 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 69

tdown 7299 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 6967, 6975, 6981, 6988, 6995, 7005, 7013, 7027, 7035, 7044, 7050, 7059, 7071, 7079, 7091, 7101, 7110, 7121, 7131, 7138, 7146, 7153, 7160, 7168, 7176, 7182, 7189, 7197, 7205, 7215, 7224, 7234, 7243, 7252, 7263, 7274, 7282, 7291, 7299]
tdown 7307 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 68

up 10236 4
tdown 10435 [622, 819, 1850, 2359, 2914, 5510, 6313, 6324, 6339, 6351, 6362, 6372, 6386, 6394, 6406, 6416, 6427, 6440, 6451, 6461, 6471, 6481, 6490, 6499, 6510, 6518, 6529, 6539, 6549, 6562, 6576, 6586, 6598, 6624, 6636, 6650, 6665, 6677, 6690, 6703, 6715, 6728, 6739, 6751, 6763, 6774, 6784, 6793, 6802, 6810, 6818, 6826, 6833, 6840, 6849, 6856, 6863, 6870, 6879, 6886, 6892, 6899, 6908, 6916, 6926, 6936, 6944, 6952, 6960, 6967, 6975, 6981, 6988, 6995, 7005, 7013, 7027, 7035, 7044, 7050, 7059, 7071, 7079, 7091, 7101, 7110, 7121, 7131, 7138, 7146, 7153, 7160, 7168, 7176, 7182, 7189, 7197, 7205, 7215, 7224, 7234, 7243, 7252, 7263, 7274, 7282, 7291, 7299, 7307, 7315, 7323, 7331, 7341, 7773, 10435]
tup 10453 [223, 416, 431, 827, 2362, 2603, 5797, 5915, 7211, 7363, 7370, 7377, 7383, 7389, 7396, 7404, 7424, 8049, 10236, 10453]
tup 10465 [223, 416, 431, 827, 2362, 2603, 5797, 5915, 7211, 7363, 7370, 7377, 7383, 7389, 7396, 7404, 7424, 8049, 10236, 10453, 10465]
OpenCV(4.6.0) D:\a\ope